# Applications of Artificial Intelligence
## SQLite Demo
### Introduction
In this notebook we'll set up the database as shown in the unit material, and then demo some of the basic queries that are possible using SQL.

### SQLite
Most database management systems work on a client-server architecture. So you might set up a machine running a system like MySQL and this machine will be in charge of managing the database on the disk – this is the sever. Then, separately, you can create one or more clients which can communicate with the server over a network. This client code could be written in Python using the [MySQL Connector](https://dev.mysql.com/doc/connector-python/en/), another similar library, or an entirely different language. The client *can* run on the same physical hardware as the server, but usually isn't, and either way, the code itself is agnostic.

This is the standard approach for most applications, especially any that have multiple users. But this requires a lot of setup just to demonstrate the basics of databases. In addition, if you are using a database within a single application then you may simply wish to bundle all of the database functionality into that one single application. This is where SQLite comes in.

SQLite allows you to write code that interfaces directly with a database that is either stored on disk (storage) or even entirely within your computer's memory, just like another variable in your code. The SQLite library becomes part of your application, rather than running on a separate server. 

If your data requirements are complicated enough to warrant going beyond flat-file formats like CSV files, but you do not need to share the data with any other users or applications, then the SQLite model might be best for your project. Of course, if you do need to interface with an existing database server, then the concepts like SQL will transfer over with some minor modifications.

### Demo
To get started, we'll import the Python library `sqlite3`.

In [2]:
import sqlite3

There are two main objects required to interface with a database using `sqlite3`. First, you create a `Connection` object, which represents the link to the database file. Then, you use this to create a `Cursor` object. The `Cursor` can execute SQL, and is used to retrieve the return values from queries.

In some applications, you may wish to create multiple cursor objects to process multiple requests with the same database at the same. Here, we will reuse a single cursor for each query.

`sqlite3.connect(filename)` connects to the file specified in the parameter and returns a `Connection` object. If you want to use an in-memory database, you can pass in `':memory:'` as the filename. If the database file does not exist, it is created.

Most changes made through queries are only temporary until the `.commit()` method is called on the connection object, at which point they will be written to the file. This means that if a sequence of queries fails (say, due to an exception), then they can be rolled back.

One exception, due to the way the underlying SQLite library works, is that a `CREATE TABLE` command will *always* automatically commit. You might find odd behaviour if you try to run the code in this notebook multiple times – you can always delete the `database.db` file to start over.

In [3]:
# .db is a common extension for sqlite databases
# though .sqlite is also used
connection = sqlite3.connect('database.db')
cursor = connection.cursor()

Once we have the cursor object, we can execute SQL commands by passing a string into the `.execute(...)` method. Let's create the tables from the examples.

On the lines below, I format the string across multiple lines to make it easier to read (using triple quoted strings, which will include the newline characters in the string). But this formatting is not required, you could write the SQL query all on one line.

In [4]:
cursor.execute("""CREATE TABLE Persons (
                   PersonID INTEGER PRIMARY KEY,
                   Name VARCHAR(255),
                   DoB DATE
                  );""")

cursor.execute("""CREATE TABLE Ratings (
                   ViewerID INTEGER NOT NULL,
                   FilmID INTEGER NOT NULL,
                   Rating INTEGER,
                   PRIMARY KEY (ViewerID, FilmID)
                  );""");

Let's insert some data into the tables.

In [5]:
cursor.execute("""INSERT INTO Persons (Name, DoB)
                  VALUES
                   ('Ulrica Arkcoll', '1966-06-11'),
                   ('Lisbeth Straw', '1979-07-01'),
                   ('Floris Redborn', '1962-01-10');""")

cursor.execute("""INSERT INTO Ratings
                  VALUES
                   (2, 26, 4),
                   (3, 79, 2),
                   (1, 79, 2),
                   (1, 26, 5);""")

connection.commit()

Now we can check to see if the data has been inserted correctly with a SELECT statement.

If you submit a query that returns one or more results to a cursor object, you can then subsequently access that data, by using `.fetchone()` to access a single row, or `.fetchall()` to access multiple.

In [6]:
cursor.execute("SELECT * FROM Ratings")
result = cursor.fetchall()

print(result)

[(2, 26, 4), (3, 79, 2), (1, 79, 2), (1, 26, 5)]


The `.execute(...)` method returns the cursor itself, and you can also use this object as an iterator in a for loop. The combination of these leads to some shortcuts which can make the code more succinct, but still readable if the query is simple:

In [7]:
for rating in cursor.execute("SELECT * FROM Ratings ORDER BY ViewerID"):
    print(f"User {rating[0]} rated film {rating[1]} as {rating[2]}/5")

User 1 rated film 26 as 5/5
User 1 rated film 79 as 2/5
User 2 rated film 26 as 4/5
User 3 rated film 79 as 2/5


You may at times be tempted to write SQL queries that pull all of the data into Python, and then simply continue to process the data from there. However, the whole point of using databases is that they have been engineered to execute queries efficiently, helping you handle data that is either too big or too complicated to manage in your application otherwise. 

Most of the time, you can do what you are trying to achieve in a single SQL query, so try to do this whenever possible, rather than falling back on Python with multiple queries. It's a good excuse to learn more SQL! 

Here's another example of a more advanced SQL query:

In [8]:
cursor.execute("""SELECT FilmID, AVG(Rating), COUNT(ViewerID)
                  FROM Ratings
                  GROUP BY FilmID
                  ORDER BY COUNT(ViewerID) DESC;""")

for result in cursor.fetchall():
    print(f"FilmID {result[0]} has an average rating of {result[1]} from {result[2]} viewers.")

FilmID 79 has an average rating of 2.0 from 2 viewers.
FilmID 26 has an average rating of 4.5 from 2 viewers.


### Exercises
Go back through this notebook, and try writing new queries in each section:

* Add a table called Films which contains FilmID, Title, and DirectorID. For an extra challenge, add the foreign key constraints into the table.
* Populate the Films table with the names of films. Make sure to include entries for films with FilmID 26 and 79.
* Modify the query in the cell above so that the output includes the names of the films, not just their IDs. You will likely want to use a [JOIN](https://www.w3schools.com/sql/sql_join.asp). Refer back to the unit material if you are unsure.